In [ ]:
import base64
import json
import os
import requests
from datetime import datetime

# 👉 Store your token safely in an environment variable before running this cell
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN", "")
if not GITHUB_TOKEN:
    raise RuntimeError("Set GITHUB_TOKEN in your environment before running this cell.")

# Encode timestamp
ts = f"Triggered at {datetime.now()}".encode("utf-8")
encoded = base64.b64encode(ts).decode("utf-8")

url = "https://api.github.com/repos/maghams62/core-api/contents/ingest_smoke.txt"

payload = {
    "message": "chore: ingest smoke test",
    "content": encoded,
    "sha": "04737cb97189ee5e36359e4b73c12203cb467cda"  # hash of empty file
}

headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}

response = requests.put(url, headers=headers, data=json.dumps(payload))
print(response.status_code, response.text)


409 {"message":"ingest_smoke.txt does not match 04737cb97189ee5e36359e4b73c12203cb467cda","documentation_url":"https://docs.github.com/rest/repos/contents#create-or-update-file-contents","status":"409"}


In [ ]:
import os
import base64
from datetime import datetime
import requests

# --- config ---
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN", "")
OWNER = "maghams62"
REPO = "core-api"
PATH = "ingest_smoke.txt"

if not GITHUB_TOKEN:
    raise RuntimeError("GITHUB_TOKEN env var is not set")

headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
}

url = f"https://api.github.com/repos/{OWNER}/{REPO}/contents/{PATH}"

# 1) Try to fetch existing file to get current SHA
resp = requests.get(url, headers=headers)
if resp.status_code == 200:
    current_sha = resp.json()["sha"]
    print(f"Existing file found, sha={current_sha}")
else:
    current_sha = None
    print(f"No existing file (status {resp.status_code}), will create new")

# 2) Build new content
ts = f"Triggered at {datetime.now()}\n".encode("utf-8")
encoded = base64.b64encode(ts).decode("utf-8")

payload = {
    "message": "chore: ingest smoke test",
    "content": encoded,
}

# Only include sha if we're updating an existing file
if current_sha:
    payload["sha"] = current_sha

# 3) PUT create/update
put_resp = requests.put(url, headers=headers, json=payload)
print(put_resp.status_code, put_resp.text)


Existing file found, sha=3fbc98f9d86ee0fc0e10caa29012ced3edb5062b
200 {"content":{"name":"ingest_smoke.txt","path":"ingest_smoke.txt","sha":"b646616e98e7d68f7e128d31ccd36078fc260d51","size":40,"url":"https://api.github.com/repos/maghams62/core-api/contents/ingest_smoke.txt?ref=main","html_url":"https://github.com/maghams62/core-api/blob/main/ingest_smoke.txt","git_url":"https://api.github.com/repos/maghams62/core-api/git/blobs/b646616e98e7d68f7e128d31ccd36078fc260d51","download_url":"https://raw.githubusercontent.com/maghams62/core-api/main/ingest_smoke.txt","type":"file","_links":{"self":"https://api.github.com/repos/maghams62/core-api/contents/ingest_smoke.txt?ref=main","git":"https://api.github.com/repos/maghams62/core-api/git/blobs/b646616e98e7d68f7e128d31ccd36078fc260d51","html":"https://github.com/maghams62/core-api/blob/main/ingest_smoke.txt"}},"commit":{"sha":"aac50a5d400a845199d07bc2fe1c3860a580654b","node_id":"C_kwDOQgy0mNoAKGFhYzUwYTVkNDAwYTg0NTE5OWQwN2JjMmZlMWMzODYwYTU4MDY1

In [ ]:
3fbc98f9d86ee0fc0e10caa29012ced3edb5062b